In [5]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def get_credentials_from_file(filename):
#     Extracts email and password from the given file. Write the file in the same directory as follows:
#     bits mail
#     mail password (google)
#     smtp mail user name
#     smtp key
#     smtp email-id   
    with open(filename, 'r') as file:


        email = file.readline().strip()
        password = file.readline().strip()
    return email, password

browser = webdriver.Chrome()

browser.get('https://onboard.bits-pilani.ac.in/ucp.php?mode=login&redirect=index.php')

# Click on Google login button using the provided HTML info
google_login_btn = browser.find_element(By.XPATH, '//a[@title="BITS Email  Login"]')
google_login_btn.click()

# Retrieve credentials from file
email, password = get_credentials_from_file('sensitive.txt')

# Find email field and fill it in
email_field = browser.find_element(By.ID, 'identifierId')
email_field.send_keys(email)
next_btn = browser.find_element(By.ID, 'identifierNext')
next_btn.click()


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Wait until password field is available and visible, then select it
password_field = WebDriverWait(browser, 10).until(
    EC.visibility_of_element_located((By.NAME, 'Passwd'))
)


# Assuming a delay, fill in the password
password_field.send_keys(password)

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import time

MAX_RETRIES = 3
retry_count = 0

while retry_count < MAX_RETRIES:
    try:
        # Wait until the button is present and clickable.
        wait = WebDriverWait(browser, 10)
        signin_btn = wait.until(EC.element_to_be_clickable((By.ID, 'passwordNext')))
        
        signin_btn.click()
        break
    except StaleElementReferenceException:
        # In case of stale element, retry after a short delay
        time.sleep(1)
        retry_count += 1
        continue
    except TimeoutException:
        retry_count += 1
        continue

if retry_count == MAX_RETRIES:
    print("Failed to click the button after maximum retries.")


try:
    element_present = EC.presence_of_element_located((By.CSS_SELECTOR, 'span.username'))
    WebDriverWait(browser, 10).until(element_present)

    # Get the current URL after signing in.
    current_url = browser.current_url
    print("Current URL:", current_url)

except TimeoutException:
    print("Timed out waiting for the new page to load after sign-in.")


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

BASE_URL = f'{current_url}&recent_topics_start='
MAX_PAGES = 10  # Adjust according to your needs

def get_notices_from_page(browser, url):
    notices = []
    
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    topics = soup.select('ul.topiclist.topics.collapsible li')
    
    for topic in topics:
        topic_element = topic.select_one('.topictitle')
        topic_title = topic_element.text.strip()
        topic_link = topic_element['href'].strip()  # Extracting the href attribute for the link
        last_post = topic.select_one('.lastpost span').text.strip()
        views = topic.select_one('.views').text.strip()
        
        notice_info = {
            'Title': topic_title,
            'Link': topic_link,  # Adding the link to the notice info dictionary
            'Last Post': last_post,
            'Views': views
        }
        
        notices.append(notice_info)
    
    return notices


all_notices = []
for i in range(0, MAX_PAGES * 5, 5):
    url = BASE_URL + str(i)
    all_notices.extend(get_notices_from_page(browser, url))

# Printing all notices
for notice in all_notices[:5]:
    print("Notices examples:")
    print("Title:", notice['Title'])
    print("Link:", notice['Link']) 
    print("Last Post:", notice['Last Post'])
    print("Views:", notice['Views'])
    print('---------------------------------------')



ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
pip install filetype


In [ ]:
from PIL import Image
from io import BytesIO
import os
import requests
from bs4 import BeautifulSoup
import filetype

BASE_URL = "https://onboard.bits-pilani.ac.in/"

def guess_extension_from_content(content):
    print(content)
    kind = filetype.guess(content)
    if kind is None:
        return ''
    return kind.extension

def sanitize_filename(filename):
    invalid_chars = ['<', '>', ':', '"', '/', '\\', '|', '?', '*']
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    return filename

def scrape_topic_content(url):
    full_url = BASE_URL + url.lstrip('./')
    
    browser.get(full_url)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    heading = soup.select_one('.topic-title').text.strip()

    author = soup.select_one('.username-coloured').text.strip()

    post_content = soup.select_one('.postbody .content')
    content_text = post_content.text.strip()

    attachments = []

    for attach_div in soup.select('.attachbox, .inline-attachment'):
        for attach in attach_div.select('.file'):
            img_tag = attach.select_one('img.postimage')
            link = img_tag['src'] if img_tag else None
            filename = attach.select_one('dd').text.split()[0]  # Assumes filename is the first word
            filesize = ' '.join(attach.select_one('dd').text.split()[1:4])  # Assumes next three words are filesize details

            attachment = {
                "filename": filename,
                "filesize": filesize,
                "link": link
            }
            attachments.append(attachment)
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    import time


    download_dir = "downloaded_attachments"
    os.makedirs(download_dir, exist_ok=True)
    downloaded_files = []

    for attach in attachments:
        browser.get(BASE_URL + attach["link"].lstrip('./'))

        time.sleep(2)

        if len(browser.find_elements(By.TAG_NAME, 'img')) > 0:
            image = browser.find_element(By.TAG_NAME, 'img')
            action = ActionChains(browser)
            action.context_click(image).send_keys(Keys.ARROW_DOWN).send_keys(Keys.RETURN).perform()

        elif len(browser.find_elements(By.CLASS_NAME, 'downloadButton')) > 0:
            download_button = browser.find_element(By.CLASS_NAME, 'downloadButton')
            download_button.click()

        time.sleep(5)

        safe_filename = sanitize_filename(attach["filename"])
        filepath = os.path.join(download_dir, safe_filename)
        downloaded_files.append(filepath)

    return {
        'heading': heading,
        'author': author,
        'content': content_text,
        'attachments': downloaded_files
    }

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os

def send_email(email_content, recipient_email, attachments=None, subject_prefix="Automated scraped email"):
    
    with open("sensitive.txt", 'r') as file:
        lines = file.readlines()
        SMTP_SERVER = "smtp.gmail.com"
        SMTP_PORT = 465
        SMTP_USERNAME = lines[2].strip()
        SMTP_PASSWORD = lines[3].strip()
        FROM_EMAIL = lines[4].strip()

    msg = MIMEMultipart()
    msg['From'] = FROM_EMAIL
    msg['To'] = recipient_email
    msg['Subject'] = f"{subject_prefix} - {email_content['heading']}"
    
    body = MIMEText(email_content['content'], 'plain', 'utf-8')
    msg.attach(body)

    if attachments:
        for file_path in attachments:
            attachment_filename = os.path.basename(file_path)
            with open(file_path, 'rb') as attachment_file:
                part = MIMEBase('application', 'octet-stream')
                part.set_payload(attachment_file.read())
                encoders.encode_base64(part)
                part.add_header('Content-Disposition', f"attachment; filename= {attachment_filename}")
                msg.attach(part)

    with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as server:
        server = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT)
        server.login(FROM_EMAIL, SMTP_PASSWORD)
        server.sendmail(FROM_EMAIL, recipient_email, msg.as_string())
    print("Sent mail..\n")
    


In [ ]:
for notice in all_notices:
scraped_content = scrape_topic_content("./viewtopic.php?f=26&t=6884")
send_email(scraped_content, "f20211457@pilani.bits-pilani.ac.in", scraped_content['attachments'])